# Longest ZigZag path in a binary tree

start from any node, then left -> right -> left -> ...

constraints
```
1<=n<=5*10**4
1<=n.val<=100
```

## dfs
```
L,R = dfs(root.left) 
...
```

In [47]:
class TreeNode:
    def __init__(self,val = 0,left = None,right = None):
        self.val = val
        self.left = left
        self.right = right
    
    @classmethod
    def construct(cls,arr):
        n = len(arr)
        def dfs(L,R):
            if L>=n:
                return [None]*(R-L+1)
            cur = [TreeNode(arr[i]) if i<n and arr[i] is not None else None for i in range(L,R+1)]
            nvalid = len([x for x in cur if x is not None])
            nxt = dfs(R+1,R+(nvalid<<1))
            i,j = 0,0
            while j<nvalid:
                while cur[i] is None:
                    i += 1
                cur[i].left = nxt[j<<1]
                cur[i].right = nxt[(j<<1)+1]
                i += 1
                j += 1
            return cur
        return dfs(0,0)[0]
    
    def __str__(self):
        ans = ''
        def bfs(q):
            nxtr = [y for x in q for y in ([x.left,x.right] if x is not None else [None,None])]
            nxt = [(None,i&1) for i in range(len(q)<<1)]
            last = all([x==None for x in nxtr])
            if not last:
                nxt = bfs(nxtr)
            n = len(nxt)>>1 
            ret = [(x+' '*(nxt[(i<<1)+1][1]+nxt[i<<1][1]-len(x)),nxt[(i<<1)+1][1]+nxt[i<<1][1]) for i in range(n) for x in [(str(q[i].val) if q[i] is not None else ' ')]]
            nonlocal ans
            tmp = ''.join([x[0] for x in ret])+'\n'
            # if not last:
            #     tmp += ''.join(['|'+'\\'+'-'*(x[1]-2) for x in ret])+'\n'
            ans = tmp+ans
            return ret
        bfs([root])
        return ans
null = None

**mistake**
- if ... else, forgot else
- (nvalid<<1) -> forgot x 2
- while arr[i] is None -> while cur[i] is None

In [48]:
root = TreeNode.construct([1,2,3,null,null,4,5,6,null,7])
print(root)

1       
2   3   
    4 5 
    6 7 



### Attempt 1

In [ ]:
class Solution:
    def longestZigZag(self,root):
        ans = 0
        def dfs(root):
            if root:
                _,R = dfs(root.left)
                L,_ = dfs(root.right)
                ans = max(ans,L,R)
                return R+1,L+1
            else:
                return 0,0
        return ans

**mistake**
- forgot to run dfs(root)
- should use self.ans

### Attempt 2

In [9]:
class Solution:
    def longestZigZag(self,root):
        self.ans = 0
        def dfs(root):
            if root:
                _,R = dfs(root.left)
                L,_ = dfs(root.right)
                self.ans = max(self.ans,L,R)
                return R+1,L+1
            else:
                return 0,0
        dfs(root)
        return self.ans

In [10]:
Solution().longestZigZag(root)

2

## optimization: just return 1?

### Attempt 3

In [ ]:
class Solution:
    def longestZigZag(self,root):
        self.ans = 0
        def dfs(root,left):
            if root:
                L,R = dfs(root.left,True),dfs(root.right,False)
                self.ans = max(self.ans,L,R)
                return R+1 if left else L+1
            else:
                return 0
        self.ans = max(self.ans,dfs(root.left,True),dfs(root.right,False))
        return self.ans

**mistake**
- wrong answer

### Attempt 4

In [ ]:
class Solution:
    def longestZigZag(self,root):
        self.ans = 0
        def dfs(root,left):
            if root:
                ret = 0
                if left:
                    dfs(root.left,True)
                    ret = dfs(root.right,False)
                else:
                    dfs(root.right,False)
                    ret = dfs(root.left,True)
                self.ans = max(self.ans,ret)
                return ret+1
            else:
                return 0
        self.ans = max(self.ans,dfs(root.left,True),dfs(root.right,False))
        return self.ans

**mistake**
- still wrong answer. wrong way of thinking it?

In [26]:
root = TreeNode.construct([1,1,1,null,1,null,null,1,1,null,1])
print(root)
Solution().longestZigZag(root)

1
1 1
x 1 x x
x x 1 1 x x x x
x x x x x 1 x x x x x x x x x x



4

### Attempt 5

In [ ]:
class Solution:
    def longestZigZag(self,root):
        self.ans = 0
        def dfs(root,left):
            # return longest path starting from root compatible with left
            # actual ans: dfs+1
            # ans will be updated for the current node
            if root:
                L,R = dfs(root.left),dfs(root.right)
                if left:
                    self.ans = max(self.ans,R+1)
                    return R+1
                else:
                    self.ans = max(self.ans,L+1)
                    return L+1
            else:
                return 0
        self.ans = max(self.ans,dfs(root.left,True)+1,dfs(root.right,False)+1)
        return self.ans

**mistake**
- forgot new parameters after refactorization

### Attempt 6

In [58]:
class Solution:
    def longestZigZag(self,root):
        self.ans = 0
        def dfs(root,left):
            if root:
                L,R = dfs(root.left,True),dfs(root.right,False)
                if left:
                    self.ans = max(self.ans,R+1)
                    print(root.val,R+1)
                    return R+1
                else:
                    self.ans = max(self.ans,L+1)
                    print(root.val,L+1)
                    return L+1
            else:
                return 0
        dfs(root.left,True)
        dfs(root.right,False)
        return self.ans

**AC**

failed attemp:

if you add 
```
self.ans = max(self.ans, dfs(root.left,True),dfs(root.right,False))
```
it will fail. At least, should be
```
L,R = dfs(root.left,True),dfs(root.right,False)
self.ans = max(self.ans,L,R)
```

But this is not necessary. dfs(root) returns the actual path length starting from root +1 assuming root has a parent. So dfs(root.left) will return path+1 starting from root.left assuming root.left has a parent. This is exactly the path starting from root, and it is included in the dfs(root.left) transverse.

In [59]:
root = TreeNode.construct([1,2,3,null,4,null,null,5,6,null,7])
print(root)
Solution().longestZigZag(root)

root = TreeNode.construct([1,2,3,null,4,null,null,5,6,null,7])
print(root)
Solution().longestZigZag(root)

1               
2       3       
    4           
    5 6         
     7          

7 1
5 2
6 1
4 3
2 4
3 1


4